In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1747206132258_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%bash
# This is a Bash cell
wget -O - https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz | aws s3 cp - s3://raw-data-emr-redfin/city_market_tracker.tsv000.gz

--2025-05-14 07:40:14--  https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz
Resolving redfin-public-data.s3.us-west-2.amazonaws.com (redfin-public-data.s3.us-west-2.amazonaws.com)... 52.218.222.33, 52.218.177.177, 52.92.180.66, ...
Connecting to redfin-public-data.s3.us-west-2.amazonaws.com (redfin-public-data.s3.us-west-2.amazonaws.com)|52.218.222.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 929621103 (887M) [binary/octet-stream]
Saving to: ‘STDOUT’

     0K .......... .......... .......... .......... ..........  0%  229K 65m56s
    50K .......... .......... .......... .......... ..........  0%  231K 65m42s
   100K .......... .......... .......... .......... ..........  0% 55.5M 43m53s
   150K .......... .......... .......... .......... ..........  0%  231K 49m15s
   200K .......... .......... .......... .......... ..........  0%  104M 39m26s
   250K .......... .......... .......... .......... .....

In [3]:
spark = SparkSession.builder.appName("RedfinDataAnalysis").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
redfin_data = spark.read.csv("s3://raw-data-emr-redfin/city_market_tracker.tsv000.gz", header=True, inferSchema=True, sep= "\t")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:

redfin_data.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------+---------------+-----------+--------------+--------+----------------------+--------------+----------+----------+----------+---------------+----------------+-----------------+---------------------+---------------------+-----------------+---------------------+---------------------+-------------+---------------+---------------+----------------+--------------------+--------------------+----------+--------------+--------------+-------------+-----------------+-----------------+------------+----------------+----------------+---------+-------------+-------------+----------------+--------------------+--------------------+----------+--------------+--------------+----------------+--------------------+--------------------+---------------+-------------------+-------------------+-----------+---------------+---------------+-----------------------+---------------------------+---------------------------+-------------------+------------------------------+--------------------+
|P

In [8]:
redfin_data.printSchema()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- PERIOD_BEGIN: date (nullable = true)
 |-- PERIOD_END: date (nullable = true)
 |-- PERIOD_DURATION: integer (nullable = true)
 |-- REGION_TYPE: string (nullable = true)
 |-- REGION_TYPE_ID: integer (nullable = true)
 |-- TABLE_ID: integer (nullable = true)
 |-- IS_SEASONALLY_ADJUSTED: boolean (nullable = true)
 |-- REGION: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- STATE_CODE: string (nullable = true)
 |-- PROPERTY_TYPE: string (nullable = true)
 |-- PROPERTY_TYPE_ID: integer (nullable = true)
 |-- MEDIAN_SALE_PRICE: string (nullable = true)
 |-- MEDIAN_SALE_PRICE_MOM: string (nullable = true)
 |-- MEDIAN_SALE_PRICE_YOY: string (nullable = true)
 |-- MEDIAN_LIST_PRICE: string (nullable = true)
 |-- MEDIAN_LIST_PRICE_MOM: string (nullable = true)
 |-- MEDIAN_LIST_PRICE_YOY: string (nullable = true)
 |-- MEDIAN_PPSF: string (nullable = true)
 |-- MEDIAN_PPSF_MOM: string (nullable = true)
 |-- MEDIAN_PPSF_YOY: string (nu

In [9]:
redfin_data.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['PERIOD_BEGIN', 'PERIOD_END', 'PERIOD_DURATION', 'REGION_TYPE', 'REGION_TYPE_ID', 'TABLE_ID', 'IS_SEASONALLY_ADJUSTED', 'REGION', 'CITY', 'STATE', 'STATE_CODE', 'PROPERTY_TYPE', 'PROPERTY_TYPE_ID', 'MEDIAN_SALE_PRICE', 'MEDIAN_SALE_PRICE_MOM', 'MEDIAN_SALE_PRICE_YOY', 'MEDIAN_LIST_PRICE', 'MEDIAN_LIST_PRICE_MOM', 'MEDIAN_LIST_PRICE_YOY', 'MEDIAN_PPSF', 'MEDIAN_PPSF_MOM', 'MEDIAN_PPSF_YOY', 'MEDIAN_LIST_PPSF', 'MEDIAN_LIST_PPSF_MOM', 'MEDIAN_LIST_PPSF_YOY', 'HOMES_SOLD', 'HOMES_SOLD_MOM', 'HOMES_SOLD_YOY', 'PENDING_SALES', 'PENDING_SALES_MOM', 'PENDING_SALES_YOY', 'NEW_LISTINGS', 'NEW_LISTINGS_MOM', 'NEW_LISTINGS_YOY', 'INVENTORY', 'INVENTORY_MOM', 'INVENTORY_YOY', 'MONTHS_OF_SUPPLY', 'MONTHS_OF_SUPPLY_MOM', 'MONTHS_OF_SUPPLY_YOY', 'MEDIAN_DOM', 'MEDIAN_DOM_MOM', 'MEDIAN_DOM_YOY', 'AVG_SALE_TO_LIST', 'AVG_SALE_TO_LIST_MOM', 'AVG_SALE_TO_LIST_YOY', 'SOLD_ABOVE_LIST', 'SOLD_ABOVE_LIST_MOM', 'SOLD_ABOVE_LIST_YOY', 'PRICE_DROPS', 'PRICE_DROPS_MOM', 'PRICE_DROPS_YOY', 'OFF_MARKET_IN_TWO_WEE

In [10]:
df_redfin = redfin_data.select(['period_end','period_duration', 'city', 'state', 'property_type',
    'median_sale_price', 'median_ppsf', 'homes_sold', 'inventory', 'months_of_supply', 'median_dom', 'sold_above_list', 'last_updated'])
df_redfin.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+----------+----------+---------------+-----------------+-------------+----------+---------+----------------+----------+---------------+--------------------+
|period_end|period_duration|      city|     state|  property_type|median_sale_price|  median_ppsf|homes_sold|inventory|months_of_supply|median_dom|sold_above_list|        last_updated|
+----------+---------------+----------+----------+---------------+-----------------+-------------+----------+---------+----------------+----------+---------------+--------------------+
|2014-11-30|             30|    Juneau| Wisconsin|All Residential|            96000| 76.923076923|         1|       16|              16|       112|              0|2025-05-12 14:39:...|
|2020-07-31|             30|Los Alamos|California|All Residential|           628000|406.841339156|         3|        6|               2|        53|   0.3333333333|2025-05-12 14:39:...|
|2022-10-31|             30|  Camptown|  Virginia|All Residential|         

In [11]:
print(f"Total number of rows: {df_redfin.count()}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total number of rows: 5860868

In [12]:

from pyspark.sql.functions import isnull
# Count null values in each column
null_counts = [df_redfin.where(isnull(col_name)).count() for col_name in df_redfin.columns]
null_counts

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
# Display the results
for i, col_name in enumerate(df_redfin.columns):
    print(f"{col_name}: {null_counts[i]} null values")

In [ ]:
# Check for missing values in the entire DataFrame
remaining_count = df_redfin.na.drop().count()

print(f"Number of missing rows: {df_redfin.count() - remaining_count}")

In [ ]:
print(f"Total number of remaining rows: {remaining_count}")

In [ ]:
#remove na and count total number of remaining rows
df_redfin = df_redfin.na.drop()
print(f"Total number of rows: {df_redfin.count()}")

In [ ]:

# Count null values in each column to confirm if we have removed all na
null_counts = [df_redfin.where(isnull(col_name)).count() for col_name in df_redfin.columns]
null_counts

In [13]:
from pyspark.sql.functions import year, month

#Extract year from period_end and save in a new column "period_end_yr"
df_redfin = df_redfin.withColumn("period_end_yr", year(col("period_end")))

#Extract month from period_end and save in a new column "period_end_month"
df_redfin = df_redfin.withColumn("period_end_month", month(col("period_end")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:

# Drop period_end and last_updated columns
df_redfin = df_redfin.drop("period_end", "last_updated")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
from pyspark.sql.functions import when

#let's map the month number to their respective month name.

df_redfin = df_redfin.withColumn("period_end_month", 
                   when(col("period_end_month") == 1, "January")
                   .when(col("period_end_month") == 2, "February")
                   .when(col("period_end_month") == 3, "March")
                   .when(col("period_end_month") == 4, "April")
                   .when(col("period_end_month") == 5, "May")
                   .when(col("period_end_month") == 6, "June")
                   .when(col("period_end_month") == 7, "July")
                   .when(col("period_end_month") == 8, "August")
                   .when(col("period_end_month") == 9, "September")
                   .when(col("period_end_month") == 10, "October")
                   .when(col("period_end_month") == 11, "November")
                   .when(col("period_end_month") == 12, "December")
                   .otherwise("Unknown")
                 )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:

df_redfin.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+----------+----------+---------------+-----------------+-------------+----------+---------+----------------+----------+---------------+-------------+----------------+
|period_duration|      city|     state|  property_type|median_sale_price|  median_ppsf|homes_sold|inventory|months_of_supply|median_dom|sold_above_list|period_end_yr|period_end_month|
+---------------+----------+----------+---------------+-----------------+-------------+----------+---------+----------------+----------+---------------+-------------+----------------+
|             30|    Juneau| Wisconsin|All Residential|            96000| 76.923076923|         1|       16|              16|       112|              0|         2014|        November|
|             30|Los Alamos|California|All Residential|           628000|406.841339156|         3|        6|               2|        53|   0.3333333333|         2020|            July|
|             30|  Camptown|  Virginia|All Residential|           200000|195.886

In [17]:
#let us write the final dataframe into our s3 bucket as a parquet file.
s3_bucket = "s3://redfin-transform-emr/redfin_data.parquet"
df_redfin.write.mode("overwrite").parquet(s3_bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…